In [1]:
#Import Statements
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time

from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC

In [2]:


#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#from selenium import webdriver
#firefox_binary = FirefoxBinary()
#driver = webdriver.Firefox(firefox_binary=firefox_binary)

#driver = webdriver.Firefox()
#driver.get("http://www.python.org")
#assert "Python" in driver.title
#elem = driver.find_element(By.NAME, "q")
#elem.clear()
#elem.send_keys("pycon")
#elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
#driver.close()

In [2]:
#Load Rochelle Data
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")
#scraped_data_frame = pd.DataFrame() !!!LETS CREATE THE DATA FRAME 
rochelle.head()


,Scientific Name,Taxon,List,GLANSIS Authors,Assessment Documented,Review by:,Shared with USGS,NOAA EndNote Library Status,Assigned to,Action Needed,...,Review Status,Internal NOAA Reviewer(s),USGS Reviewer?,Past External Reviewers,New External Reviewers,Review complete and Cleared to post,Date Profile Last Updated (Posted online),Last Review Date (GLANSIS Records),OIA/RA Posted to Clearinghouse,Twitter
0,Cabomba caroliniana,Plants (S),Established,"Larson, Cao, Berent, Iott",TM161,2021.0,Y,5 year - Received 8/9/2022,EL,None,...,NaN,RS,NaN,Gannon,NaN,NaN,2016-07-25 00:00:00,NaN,NaN,NaN
1,Myriophyllum spicatum,Plants (S),Established,Berent,TM161,2021.0,Y,5 year delivered 3/4/2022,EL,add link to sibiricum x spicatum. Check refer...,...,NaN,NaN,NaN,Dennis Albert,NaN,NaN,2016-03-21 00:00:00,NaN,NaN,NaN
2,Phragmites australis,Plants (E),Established,"Sturtevant, Fusaro, Conard, Iott",TM161b,2021.0,Y,NaN,El,None,...,NaN,NaN,NaN,S. Galatowitsch,NaN,NaN,Complete,NaN,NaN,NaN
3,Nymphoides peltata,Plants (S),Established,Berent,TM161,2021.0,Y,5 year delivered 8/19/2022,EL,Check references are in NAS.,...,NaN,NaN,NaN,Selheimer,NaN,NaN,2016-03-23 00:00:00,NaN,NaN,NaN
4,Potamogeton crispus,Plants (S),Established,"Cao, Berent",TM161,2021.0,Y,5 year delivered 9/13/2022,EL,Check references are in NAS.,...,NaN,NaN,NaN,Selheimer,NaN,NaN,2016-03-18 00:00:00,NaN,NaN,NaN


In [3]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(namelist)

['Cabomba caroliniana', 'Myriophyllum spicatum', 'Phragmites australis', 'Nymphoides peltata', 'Potamogeton crispus', 'Trapa natans', 'Corbicula fluminea', 'Gambusia affinis', 'Pimephales vigilax', 'Crassula helmsii', 'Pistia stratiotes', 'Stratiotes aloides', 'Veronica beccabunga', 'Rhithropanopeus harrisi', 'Thermocyclops crassus', 'Cherax destructor', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lepomis auritus', 'Limnoperna fortunei', 'Alnus glutinosa', 'Apeltes quadracus', 'Arundo donax', 'Branchiura sowerbyi', 'Channa argus', 'Cipangopaludina japonica', 'Cyclops strenuus', 'Didymosphenia geminata', 'Discostella (Cyclotella) pseudostelligera', 'Leuciscus idus', 'Neoergasilus japonicus', 'Procambarus clarkii', 'Psammonobiotus communis', 'Psammonobiotus dziwnowi', 'Psammonobiotus linearis', 'Syngnathus abaster', 'Agrostis gigantea', 'Butomus umbellatus', 'Carassius carassius', 'Carex acutiformis', 'Carex disticha', 'Chenopodium glaucum (Oxybasis glauca)', 'Cirsium p

In [4]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [5]:
#accessing search page of the herbarium
herbarium_url = "https://quod.lib.umich.edu/cgi/i/image/image-idx?xc=1;page=searchgroup;g=herb-ic"
herbarium_html = urllib.request.urlopen(herbarium_url, context=ctx).read()
#herbarium_html


In [10]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
#send driver to the herbarium website and submit a search query and scrape the amount of results
#BE SURE TO SWITCH ALL "FIND ELEMENTS" TO XPATH IF PLAUSIBLE?
for speciesname in namelist[:]:
    driver.get(herbarium_url) # Goes to herbarium
    elem = driver.find_element(by = By.NAME, value = 'q1') #Q1 is the first search box?
    elem.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton = driver.find_element(by = By.CSS_SELECTOR, value= '#collection-search > button.\[.button.button--cta.\]') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    searchbutton.click()
    #search_results_url = driver.current_url
    try:
        try:
            resultcount = driver.find_element(by = By.CLASS_NAME, value ='results-heading').text #an error that can't execute this occurs when i restart running the data
        except:
            try:
                resultcount = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/div[1]/div/h2')
            except:
                try:
                    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div[2]/div[2]/div[1]/div/h2'))).click()
                except:
                    WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.XPATH, '/html/body/main/div[2]/div[2]/div[1]/div/h2')))
                    driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div[2]/div[2]/div[1]/div/h2')))) #SHOULD THIS HAVE A CLICK?

        #next line returns a list of two numbers like ['23','23']
        resultcountnumberlist= list(map(int, list(filter(lambda x: x!= '', re.findall(r'\d*', re.search(r'\d* of \d*', resultcount).group()))))) #first number in list is the total results displayed on the page, the second number is the total results

    except AttributeError: #I think this was for if there are no results?
        continue # should break out of this loop and continue to the next species name
    try:
        resultitem = driver.find_element(by=By.XPATH, value='/html/body/main/div[2]/div[2]/section[1]/a')
        resultitem.click()
    except: #this except block might be unnecessary now that the problem was solved with making the window full screen
        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div[2]/div[2]/section[1]/a'))).click()
        except:
            WebDriverWait(driver, 20).until(EC.invisibility_of_element((By.XPATH, '/html/body/main/div[2]/div[2]/section[1]/a')))
            driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/main/div[2]/div[2]/section[1]/a')))) #SHOULD THIS HAVE A CLICK?
    #individually select each search result and scrape each matching entry

    #driver2 = webdriver.Chrome()
    
    for result in range(int(resultcountnumberlist[1])): #[:-1]: #properly traverses the amount of results on the page
        scrapeddata1 = []
        scrapeddata2 = []
        time.sleep(2) #sleep allows the servers time to communicate
        resp = requests.get(driver.current_url)
        #print(resp.content)
        newsoup = BeautifulSoup(resp.content, 'html.parser')
        collection_category = newsoup.find_all('dt')
        collection_data = newsoup.find_all('dd')
        for tag in collection_category:
            scrapeddata1.append(tag.text)
        for tag in collection_data:
            scrapeddata2.append(tag.text)
        try:
            nextbutton = driver.find_element(by=By.XPATH, value='/html/body/main/div[1]/p/a[2]')
            nextbutton.click()
        except:
            continue
        scraped_data = dict(zip(scrapeddata1, scrapeddata2)) #scraped data includes column names and data
        result_df = pd.DataFrame(data=scraped_data, index=[0])
        #print(result_df)
        if scraped_dataframe.empty:
            scraped_dataframe = result_df
            #print('Empty')
        else:
            scraped_dataframe = scraped_dataframe.append(pd.DataFrame(data = result_df))
            #print('Not Empty Anymore')
        #print(result_df)
    #scraped_dataframe.append(result_df)



In [12]:
#scraped_dataframe.columns
print(resultcountnumberlist)
file= scraped_dataframe.to_csv(r'C:\Users\justi\Downloads\herbarium1-out.csv', encoding='utf-8')

[11, 11]


In [6]:
sdf = pd.read_csv(r"C:\Users\justi\Downloads\herbarium-final scrape full.csv")

In [7]:
sdf.head()

,Unnamed: 0,Decimal Latitude,Decimal Longitude,Catalog Number,Scientific Name,Family,Identified By,Collector,Collector Number,Event Date,...,Publication,Genus,Species,Scale,Herbarium Number,Collection Location,Collection Date,Notes Alert,Creator,General Information
0,0,44.661740,-77.917490,1201461.0,\nCabomba caroliniana\n,Cabombaceae,Michael J. Oldham,Michael J. Oldham,22526,1999-07-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,44.637690,-77.942760,1201476.0,\nCabomba caroliniana\n,Cabombaceae,M.J. Oldham & D.A. Sutherland,Michael J. Oldham,22414,1999-07-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,42.777630,-83.584530,1302380.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Anton A. Reznicek,10329,1996-09-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,42.089547,-85.539805,1302381.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Fred W. Rapp & Clarence R. Hanes,1446,1936-08-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NaN,NaN,1302382.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Clarence R. Hanes,S.J. 171,1954-09-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
sdf.drop('Unnamed: 0', axis=1)

,Decimal Latitude,Decimal Longitude,Catalog Number,Scientific Name,Family,Identified By,Collector,Collector Number,Event Date,Continent,...,Publication,Genus,Species,Scale,Herbarium Number,Collection Location,Collection Date,Notes Alert,Creator,General Information
0,44.661740,-77.917490,1201461.0,\nCabomba caroliniana\n,Cabombaceae,Michael J. Oldham,Michael J. Oldham,22526,1999-07-19,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44.637690,-77.942760,1201476.0,\nCabomba caroliniana\n,Cabombaceae,M.J. Oldham & D.A. Sutherland,Michael J. Oldham,22414,1999-07-12,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42.777630,-83.584530,1302380.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Anton A. Reznicek,10329,1996-09-04,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,42.089547,-85.539805,1302381.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Fred W. Rapp & Clarence R. Hanes,1446,1936-08-21,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1302382.0,\nCabomba caroliniana\n,Cabombaceae,NaN,Clarence R. Hanes,S.J. 171,1954-09-28,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5310,NaN,NaN,1577545.0,\nSalvinia minima\n,Salviniaceae,NaN,"D. Kinbacher, J. Ross, & William L. McCart",10180,1968-11-10,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5311,NaN,NaN,1577546.0,\nSalvinia minima\n,Salviniaceae,NaN,Wilbur H. Duncan,20111,1956-05-26,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5312,NaN,NaN,1577575.0,\nSalvinia minima\n,Salviniaceae,NaN,F. D. Barlow,28/21,1963-05-16,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5313,NaN,NaN,1577576.0,\nSalvinia minima\n,Salviniaceae,NaN,D. E. Breedlove,56065,1981-12-12,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
